In [1]:
import math
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from datetime import datetime
from time import strftime, localtime
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import gensim.corpora as corpora
from pprint import pprint
from wordcloud import WordCloud
import os
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pickle 
from gensim.models.ldamulticore import LdaMulticore
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

input_file = r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\twitter data\Tweets about the Top Companies from 2015 to 2020\Tweet.csv\Tweet.csv"
input_df  = pd.read_csv(input_file, index_col="tweet_id")

#TCT = pd.read_csv(r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\twitter data\Tweets about the Top Companies from 2015 to 2020\Company_Tweet.csv\Company_Tweet.csv", index_col="tweet_id")
#TT  = pd.read_csv(r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\twitter data\Tweets about the Top Companies from 2015 to 2020\Tweet.csv\Tweet.csv", index_col="tweet_id")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fabio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Main
def import_twitter_data_period(target_file, filters_dict, start_datetime, finish_datetime):
    #prep data
    input_df = pd.read_csv(target_file)
    epoch_time  = datetime(1970, 1, 1)
    epoch_start = (start_datetime - epoch_time).total_seconds()
    epoch_end   = (finish_datetime - epoch_time).total_seconds()
    if filters_dict != None and filters_dict != dict():
        raise ValueError("filters_dict input not programmed yet")
    
    #trim according to time window    
    input_df = input_df[input_df["post_date"]>epoch_start]
    input_df = input_df[input_df["post_date"]<epoch_end]
    
    return input_df


def update_shortened_company_file(df_stocks_list_file, corp_stopwords, file_location=None):
    stocks_list         = list(df_stocks_list_file["Name"].map(lambda x: x.lower()).values)
            
    stocks_list_shortened_dict = dict()
    for stock_name in stocks_list:
        shortened = False
        stock_name_split = stock_name.split(" ")
        for word in reversed(stock_name_split):
            for stop in corp_stopwords:
                if stop == word:
                    stock_name_split.remove(word)
                    shortened = True
        if shortened == True:
            stocks_list_shortened_dict[stock_name] = " ".join(stock_name_split)
    
    return stocks_list_shortened_dict


def prep_twitter_text_for_subject_discovery(input_list, df_stocks_list_file=None):
    #prep parameters
    death_characters    = ["$", "amazon", "apple", "goog", "tesla", "http", "@", "1", "2", "3", "4", "5", "6", "7", "8", "9", "0", ".", "'s", "compile", "www"]
    stocks_list         = list(df_stocks_list_file["Name"].map(lambda x: x.lower()).values)
    tickers_list        = list(df_stocks_list_file["Ticker"].map(lambda x: x.lower()).values)
    stopwords_english   = stopwords.words('english')
    #these are words are removed from company names to create additional shortened versions of those names. This is so these version can be eliminated from the tweets to make the subjects agnostic
    corp_stopwords      = [".com", "company", "corp", "froup", "fund", "gmbh", "global", "incorporated", "inc.", "inc", "tech", "technology", "technologies", "trust", "limited", "lmt", "ltd"]
    #these are words are directly removed from tweets
    misc_stopwords      = ["iphone", "airpods", "jeff", "bezos", "#microsoft", "#amzn", "volkswagen", "microsoft", "amazon's", "tsla", "androidwear", "ipad", "amzn", "iphone", "tesla", "TSLA", "elon", "musk", "baird", "robert", "pm", "androidwear", "android", "robert", "ab", "ae", "dlvrit", "https", "iphone", "inc", "new", "dlvrit", "py", "twitter", "cityfalconcom", "aapl", "ing", "ios", "samsung", "ipad", "phones", "cityfalconcom", "us", "bitly", "utmmpaign", "etexclusivecom", "cityfalcon", "owler", "com", "stock", "stocks", "buy", "bitly", "dlvrit", "alexa", "zprio", "billion", "seekalphacom", "et", "alphet", "seekalpha", "googl", "zprio", "trad", "jt", "windows", "adw", "ifttt", "ihadvfn", "nmona", "pphppid", "st", "bza", "twits", "biness", "tim", "ba", "se", "rat", "article"]


    #prep stocks_list_shortened
    stocks_list_shortened_dict  = update_shortened_company_file(df_stocks_list_file, corp_stopwords)
    stocks_list_shortened       = list(stocks_list_shortened_dict.values)
    
    #prep variables
    split_tweets = []
    output = []
    for tweet in input_list:
        split_tweet_pre = tweet.split(" ")
        split_tweet = []
        for word in split_tweet_pre:
            #split_tweet = split_tweet + [" " + word.lower() + " "]
            split_tweet = split_tweet + [word.lower()]
        split_tweets = split_tweets + [split_tweet]

    #removal of words
    for tweet in split_tweets:
        for word in reversed(tweet):
            Removed = False
            # remove words containing "x"
            for char in death_characters:
                if char in word:
                    tweet.remove(word)
                    Removed = True
                    break
            if Removed == False:
                for char in tickers_list + stopwords_english + corp_stopwords + misc_stopwords:
                    if char == word:
                        tweet.remove(word)
                        S = False
                        break
            # remove words equalling stop words
        
        
    #finalise and remove stock names
    output = []
    iteration_list = list(reversed(stocks_list)) + list(reversed(stocks_list_shortened))
    for split_tweet in split_tweets:
        #recombined_tweet = list(map(lambda x: x.strip(), split_tweet))
        recombined_tweet = " ".join(split_tweet)#.replace("  "," ")
        #recombined_tweet = " ".join(recombined_tweet)#.replace("  "," ")
        for stock_name in iteration_list:
            recombined_tweet = recombined_tweet.replace(stock_name, "")
        output = output + [recombined_tweet]
    
    return output

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))


def return_subject_keys(df_prepped_tweets_company_agnostic, topic_qty = 10, enforced_topics_dict=None, stock_names_list=None, words_to_remove = None, 
                        return_LDA_model=True, return_png_visualisation=False, return_html_visualisation=False, 
                        alpha=0.1, apply_IDF=True, cores=2):
    output = []

    data = df_prepped_tweets_company_agnostic
    data_words = list(sent_to_words(data))
    if return_LDA_model < return_html_visualisation:
        raise ValueError("You must return the LDA visualisation if you return the LDA model")

       
    if return_png_visualisation==True:
        long_string = "start"
        for w in data_words:
            long_string = long_string + ',' + ','.join(w)
        wordcloud = WordCloud(background_color="white", max_words=1000, contour_width=3, contour_color='steelblue')
        wordcloud.generate(long_string)
        wordcloud.to_image()
        output = output + [wordcloud]
    else:
        output = output + [None]
    
    if return_LDA_model==True:
        # Create Dictionary
        id2word = corpora.Dictionary(data_words)

        # Create Corpus
        texts = data_words

        # Term Document Frequency
        corpus = [id2word.doc2bow(text) for text in texts]

        #translate the enforced_topics_dict input
        eta = None
        if enforced_topics_dict != None:
            eta = np.zeros(len(id2word))
            offset = 1
            for group_num in range(len(enforced_topics_dict)):
                for word in enforced_topics_dict[group_num]:
                    try: 
                        word_id = id2word.token2id[word]
                        eta[word_id] = group_num + offset
                    except:
                        a=1

        #apply IDF
        if apply_IDF == True:
            # create tfidf model
            tfidf = TfidfModel(corpus)

            # apply tfidf to corpus
            corpus = tfidf[corpus]
        
        
        
        # Build LDA model
        lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=topic_qty,
                                               eta=eta,
                                               alpha = alpha, # controls topic sparsity
                                               #beta = beta, # controls word sparsity
                                               workers=cores)
        
        # Print the Keyword in the 10 topics
        #pprint(lda_model.print_topics())
        doc_lda = lda_model[corpus]
        output = output + [lda_model, doc_lda, corpus, id2word]
    else:
        output = output + [None, None, None]
        
        
    if return_html_visualisation==True:
        pyLDAvis.enable_notebook()
        LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
        output = output + [LDAvis_prepared]
    else:
        output = output + [None]
    
    return tuple(output)

def save_load_subject_keys(output_folder_name, run_name, wordcloud=None, doc_lda=None, corpus=None, id2word=None, visualisation=None):
    #run_name = "test"
    if not os.path.isdir(output_folder_name):
    #os.path.dirname(os.path.dirname(path))
        os.mkdir(output_folder_name)
    
    if wordcloud != None:
        wordcloud.to_file(output_folder_name + "word cloud_" + run_name + ".png")

    #if doc_lda != None or corpus != None or id2word != None:
    #    
    #    try:
    #        LDAvis_prepared = gensimvis.prepare(doc_lda, corpus, id2word)
    #        with open(LDAvis_data_filepath, 'wb') as f:
    #            pickle.dump(LDAvis_prepared, f)
    #    except:
    #        print("error, outputs doc_lda, corpus & id2word are not yet programmed to save")
        
    if visualisation != None:
        pyLDAvis.save_html(visualisation, output_folder_name + "word cloud_" + run_name +'.html')

def export_words_within_topics_count(lda_model):
    strings_dict = dict()
    for topic in lda_model.print_topics():
        tuples = topic[1].split("+")
        for tup in tuples:
            word = tup.split("+")[0].split("*")[1]
            word = re.sub('[^a-zA-Z]', '', word)
            num  = tup.split("+")[0].split("*")[0]
            if not word in strings_dict.keys():
                strings_dict[word] = 0
            strings_dict[word] += float(num)

    words_within_topics_count = pd.DataFrame(list(strings_dict.items()), columns=['keys', 'values'])
    words_within_topics_count.sort_values(by="values", ascending=False, inplace=True)
    model_time = datetime.now()
    model_time = model_time.strftime("%Y%m%d_%H%M")
    words_within_topics_count.to_csv("C:\\Users\\Fabio\\OneDrive\\Documents\\Studies\\Final Project\\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\\test books\\output\\word_topics" + model_time + ".csv")


#parameters
tweet_qty = int(4e3)
topic_qty = 7
alpha=0.1


input_file = r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\twitter data\Tweets about the Top Companies from 2015 to 2020\Tweet.csv\Tweet.csv"
start_datetime = datetime.strptime('01/01/17 00:00:00', '%m/%d/%y %H:%M:%S')
end_datetime = datetime.strptime('01/01/18 00:00:00', '%m/%d/%y %H:%M:%S')
enforced_topics_dict = [
    ['investment', 'financing', 'losses'],
    ['risk', 'exposure', 'liability'],
    ["financial forces" , "growth", "interest rates"]]
df_stocks_list_file     = pd.read_csv(r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\data\stock_info.csv")
output_folder_name = "C:\\Users\\Fabio\\OneDrive\\Documents\\Studies\\Final Project\\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\\test books\\output\\"


#data prep
#print(datetime.now().strftime("%H:%M:%S"))
df_prepped_tweets                                               = import_twitter_data_period(input_file, dict(), start_datetime, end_datetime)
#print("--------------------------------1--------------------------------")
#print(datetime.now().strftime("%H:%M:%S"))
df_prepped_tweets_company_agnostic                              = prep_twitter_text_for_subject_discovery(df_prepped_tweets["body"][:tweet_qty], df_stocks_list_file=df_stocks_list_file)
#print("--------------------------------2--------------------------------")
#print(datetime.now().strftime("%H:%M:%S"))
wordcloud, lda_model, doc_lda, corpus, id2word, visualisation   = return_subject_keys(df_prepped_tweets_company_agnostic, topic_qty = topic_qty, alpha=alpha, apply_IDF=True,
                                                                                     enforced_topics_dict=enforced_topics_dict, return_LDA_model=True, return_png_visualisation=True, return_html_visualisation=True)
#print("--------------------------------3--------------------------------")
#print(datetime.now().strftime("%H:%M:%S"))
save_load_subject_keys(output_folder_name, str(tweet_qty), wordcloud=wordcloud, doc_lda=doc_lda, corpus=corpus, id2word=id2word, visualisation=visualisation)
export_words_within_topics_count(lda_model)
#print("--------------------------------4--------------------------------")
#print(datetime.now().strftime("%H:%M:%S"))


c:\Users\Fabio\AppData\Local\Programs\Python\Python311\Lib\site-packages\wordcloud\wordcloud.py:106: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  self.colormap = plt.cm.get_cmap(colormap)


In [3]:
for index in df_prepped_tweets.index[:60]:
    test_tweet = df_prepped_tweets["body"][index].split(" ")
    bow_doc = id2word.doc2bow(test_tweet)
    doc_topics = lda_model.get_document_topics(bow_doc)
    print(doc_topics)




[(0, 0.14285715), (1, 0.14285715), (2, 0.14285715), (3, 0.14285715), (4, 0.14285715), (5, 0.14285715), (6, 0.14285715)]
[(0, 0.013031844), (1, 0.15157343), (2, 0.013027253), (3, 0.5531987), (4, 0.013029001), (5, 0.013028525), (6, 0.24311128)]
[(0, 0.021308025), (1, 0.34789196), (2, 0.021307442), (3, 0.021309627), (4, 0.021307878), (5, 0.021306653), (6, 0.54556847)]
[(0, 0.4820208), (1, 0.03705436), (2, 0.037058562), (3, 0.037057955), (4, 0.3326843), (5, 0.037066404), (6, 0.037057567)]
[(0, 0.013025202), (1, 0.0130317565), (2, 0.01303112), (3, 0.37799293), (4, 0.18541045), (5, 0.013024891), (6, 0.38448364)]
[(0, 0.02708964), (1, 0.027081376), (2, 0.41057363), (3, 0.027085837), (4, 0.45399415), (5, 0.027091347), (6, 0.02708402)]
[(0, 0.01493935), (1, 0.5545699), (2, 0.014935675), (3, 0.014941755), (4, 0.014935872), (5, 0.3707381), (6, 0.014939351)]
[(0, 0.013001588), (1, 0.16419628), (2, 0.013001227), (3, 0.013002757), (4, 0.17917131), (5, 0.013001596), (6, 0.6046252)]
[(0, 0.014934608),

In [4]:
print("Hee")
pred_output_and_tickers_combos_list = [("aapl", "<CLOSE>"), ("carm", "<HIGH>")]

Hee


In [ ]:
def filter_tweets_for_company(df_prepped_tweets, company):
    
    
    
    
    print("hello")

def return_topic_company_sentiment(df_prepped_sentiment, sentiment_model, pred_output_and_tickers_combos_list, sentiment_method, topic_model, period_start, period_end, seconds_per_time_steps, relavance_halflife, relavance_lifetime, weighted_topic_relavance = False):
    df_sentiment_data_prepped = pd.DataFrame()
    
    #list unique companies
    companies_list = set()
    for output_and_tickers in pred_output_and_tickers_combos_list:
        companies_list.add(output_and_tickers[0])
    companies_list = list(companies_list)
    
    
    for company in companies_list:
        df_tweets_company = filter_tweets_for_company(df_prepped_tweets, company)
        #df_tweets_company = annotate_tweets_topics(df_tweets_company, sentiment_method, topic_model)
        #df_sentiment_company_single = return_sentiment_company(df_tweets_company, period_start, period_end, seconds_per_time_steps, relavance_halflife, relavance_lifetime, weighted_topic_relavance = False)
        #df_sentiment_data_prepped = pd.concat([df_sentiment_data_prepped, df_sentiment_company_single])
    return df_sentiment_data_prepped

df_prepped_tweets       = df_prepped_tweets
sentiment_model_dict    = {
    "lda_model" : lda_model,
    "doc_lda"   : doc_lda,
    "corpus"    : corpus,
    "id2word"   : id2word
}
pred_output_and_tickers_combos_list = [("aapl", "<CLOSE>"), ("carm", "<HIGH>")]
df_prepped_tweets       = df_prepped_tweets
companies_list          = None
sentiment_method        = None
topic_model             = None
period_start            = None
period_end              = None
seconds_per_time_steps  = None
relavance_halflife      = None
relavance_lifetime      = None

df_sentiment_data_prepped = return_topic_company_sentiment(df_prepped_tweets, sentiment_model_dict, pred_output_and_tickers_combos_list, sentiment_method, topic_model, period_start, period_end, seconds_per_time_steps, relavance_halflife, relavance_lifetime, weighted_topic_relavance = False)




Mathematical Models

1. 


